In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
import os
import spotipy
import pandas as pd
from tqdm import tqdm
%matplotlib inline

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials
cid = 'b9ff596f8afd419ab00f96c0e3ff1aff'
secret = 'fcc60a813705409090478d22581c726e'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [41]:
track_results

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=genre%3Apop&type=track&offset=990&limit=5',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2W8yFh0Ga6Yf3jiayVxwkE'},
       'href': 'https://api.spotify.com/v1/artists/2W8yFh0Ga6Yf3jiayVxwkE',
       'id': '2W8yFh0Ga6Yf3jiayVxwkE',
       'name': 'Dove Cameron',
       'type': 'artist',
       'uri': 'spotify:artist:2W8yFh0Ga6Yf3jiayVxwkE'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
 

In [66]:
artist_name = []
track_name = []
popularity = []
track_id = []
release_date = []
for year in range(2000, 2024):
    for i in range(0, 1000, 50):
        track_results = sp.search(q='genre:pop; year:' + str(year), type='track', limit=50, offset=0, market='US')
        for index, t in enumerate(track_results['tracks']['items']):
            artist_name.append(t['artists'][0]['name'])
            track_name.append(t['name'])
            track_id.append(t['id'])
            popularity.append(t['popularity'])
            release_date.append(t['album']['release_date'])
    

In [68]:
len(release_date)

24000

In [69]:
track_dataframe = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'popularity' : popularity, 'release_date':release_date })
print(track_dataframe.shape)
track_dataframe.head()

(24000, 5)


,artist_name,track_name,track_id,popularity,release_date
0,Coldplay,Yellow,3AJwUDP919kvQ9QcozQPxg,85,2000-07-10
1,3 Doors Down,Kryptonite,6ZOBP3NvffbU4SZcrnt1k6,76,2000
2,Outkast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,78,2000-10-31
3,Shaggy,It Wasn't Me,3WkibOpDF7cQ5xntM1epyf,75,2000-08-08
4,Nelly,Ride Wit Me,3Gf5nttwcX9aaSQXRWidEZ,72,2000-06-27


In [70]:
track_dataframe.sample(5)

,artist_name,track_name,track_id,popularity,release_date
23394,Justin Bieber,Yummy,4IuHVTVbBaLhiwgmgzEl7q,0,2023-02-11
23475,Ed Sheeran,Shivers,5sD0PQoU5NN4hpiKH5osQ0,0,2023-02-10
18684,Summer Walker,Karma,2Fyjjpg03fn7n5cj0Qm380,72,2018-10-19
8480,Britney Spears,Circus,7jk7gqyEonmVVYahZN5zhW,71,2008-12-02
15336,Drake,Jungle,7JXZq0JgG2zTrSOAgY8VMC,71,2015-02-12


In [71]:
track_dataframe

,artist_name,track_name,track_id,popularity,release_date
0,Coldplay,Yellow,3AJwUDP919kvQ9QcozQPxg,85,2000-07-10
1,3 Doors Down,Kryptonite,6ZOBP3NvffbU4SZcrnt1k6,76,2000
2,Outkast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,78,2000-10-31
3,Shaggy,It Wasn't Me,3WkibOpDF7cQ5xntM1epyf,75,2000-08-08
4,Nelly,Ride Wit Me,3Gf5nttwcX9aaSQXRWidEZ,72,2000-06-27
...,...,...,...,...,...
23995,Justin Bieber,Anyone,0RYu8aisEwtFc3SEIw68he,0,2023-02-13
23996,Justin Bieber,Ghost,56957v8LMKDlcrisX2kp7b,0,2023-02-11
23997,Justin Bieber,Holy,1j1Bn7kWnPWck79XYSTHI5,0,2023-02-10
23998,Justin Bieber,Beautiful Love - Free Fire,0Y1flwKfQc97a92yP7VbeW,0,2023-02-10


In [72]:
features_df = pd.DataFrame()
for i in tqdm(track_id):
    features_i = sp.audio_features(tracks=[i])
    features_df_i = pd.DataFrame(features_i)
    features_df = pd.concat([features_df, features_df_i])

100%|██████████| 24000/24000 [37:49<00:00, 10.57it/s]  


In [73]:
features_df.reset_index(inplace=True)

In [74]:
features_df.sample(5)

,index,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
19183,0,0.715,0.624,4,-3.046,0,0.1140,0.11000,0.000000,0.1230,0.412,158.087,audio_features,3HWzoMvoF3TQfYg4UPszDq,spotify:track:3HWzoMvoF3TQfYg4UPszDq,https://api.spotify.com/v1/tracks/3HWzoMvoF3TQ...,https://api.spotify.com/v1/audio-analysis/3HWz...,173325,4
14122,0,0.885,0.581,11,-8.813,1,0.0378,0.42700,0.000204,0.0759,0.728,114.016,audio_features,6vECYJHxYmm3Ydt3fF01pE,spotify:track:6vECYJHxYmm3Ydt3fF01pE,https://api.spotify.com/v1/tracks/6vECYJHxYmm3...,https://api.spotify.com/v1/audio-analysis/6vEC...,313684,4
2680,0,0.431,0.950,7,-4.918,1,0.2600,0.00105,0.000036,0.2720,0.560,191.936,audio_features,2OMqSRS11ly56zg9u5za6c,spotify:track:2OMqSRS11ly56zg9u5za6c,https://api.spotify.com/v1/tracks/2OMqSRS11ly5...,https://api.spotify.com/v1/audio-analysis/2OMq...,158733,4
12381,0,0.670,0.609,7,-10.088,1,0.0333,0.74900,0.000067,0.2270,0.573,127.463,audio_features,3Hvg5tRKsQlX25wYwgMF9p,spotify:track:3Hvg5tRKsQlX25wYwgMF9p,https://api.spotify.com/v1/tracks/3Hvg5tRKsQlX...,https://api.spotify.com/v1/audio-analysis/3Hvg...,109736,4
22773,0,0.848,0.512,3,-7.720,0,0.1020,0.22800,0.002320,0.0879,0.572,120.009,audio_features,2wSTnntOPRi7aQneobFtU4,spotify:track:2wSTnntOPRi7aQneobFtU4,https://api.spotify.com/v1/tracks/2wSTnntOPRi7...,https://api.spotify.com/v1/audio-analysis/2wST...,181840,4


In [75]:
concat_df = pd.concat([track_dataframe, features_df], axis = 1)

In [76]:
concat_df

,artist_name,track_name,track_id,popularity,release_date,index,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Coldplay,Yellow,3AJwUDP919kvQ9QcozQPxg,85,2000-07-10,0,0.429,0.661,11,-7.227,...,0.2340,0.285,173.372,audio_features,3AJwUDP919kvQ9QcozQPxg,spotify:track:3AJwUDP919kvQ9QcozQPxg,https://api.spotify.com/v1/tracks/3AJwUDP919kv...,https://api.spotify.com/v1/audio-analysis/3AJw...,266773,4
1,3 Doors Down,Kryptonite,6ZOBP3NvffbU4SZcrnt1k6,76,2000,0,0.545,0.865,11,-5.708,...,0.1680,0.543,99.009,audio_features,6ZOBP3NvffbU4SZcrnt1k6,spotify:track:6ZOBP3NvffbU4SZcrnt1k6,https://api.spotify.com/v1/tracks/6ZOBP3NvffbU...,https://api.spotify.com/v1/audio-analysis/6ZOB...,233933,4
2,Outkast,Ms. Jackson,0I3q5fE6wg7LIfHGngUTnV,78,2000-10-31,0,0.843,0.806,4,-5.946,...,0.0771,0.613,94.948,audio_features,0I3q5fE6wg7LIfHGngUTnV,spotify:track:0I3q5fE6wg7LIfHGngUTnV,https://api.spotify.com/v1/tracks/0I3q5fE6wg7L...,https://api.spotify.com/v1/audio-analysis/0I3q...,270507,4
3,Shaggy,It Wasn't Me,3WkibOpDF7cQ5xntM1epyf,75,2000-08-08,0,0.853,0.606,0,-4.596,...,0.3130,0.654,94.759,audio_features,3WkibOpDF7cQ5xntM1epyf,spotify:track:3WkibOpDF7cQ5xntM1epyf,https://api.spotify.com/v1/tracks/3WkibOpDF7cQ...,https://api.spotify.com/v1/audio-analysis/3Wki...,227600,4
4,Nelly,Ride Wit Me,3Gf5nttwcX9aaSQXRWidEZ,72,2000-06-27,0,0.850,0.700,7,-6.490,...,0.2440,0.722,101.875,audio_features,3Gf5nttwcX9aaSQXRWidEZ,spotify:track:3Gf5nttwcX9aaSQXRWidEZ,https://api.spotify.com/v1/tracks/3Gf5nttwcX9a...,https://api.spotify.com/v1/audio-analysis/3Gf5...,291782,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,Justin Bieber,Anyone,0RYu8aisEwtFc3SEIw68he,0,2023-02-13,0,0.686,0.538,2,-8.026,...,0.1130,0.584,115.884,audio_features,0RYu8aisEwtFc3SEIw68he,spotify:track:0RYu8aisEwtFc3SEIw68he,https://api.spotify.com/v1/tracks/0RYu8aisEwtF...,https://api.spotify.com/v1/audio-analysis/0RYu...,190779,4
23996,Justin Bieber,Ghost,56957v8LMKDlcrisX2kp7b,0,2023-02-11,0,0.604,0.741,2,-5.571,...,0.4090,0.473,153.947,audio_features,56957v8LMKDlcrisX2kp7b,spotify:track:56957v8LMKDlcrisX2kp7b,https://api.spotify.com/v1/tracks/56957v8LMKDl...,https://api.spotify.com/v1/audio-analysis/5695...,153190,4
23997,Justin Bieber,Holy,1j1Bn7kWnPWck79XYSTHI5,0,2023-02-10,0,0.673,0.704,6,-8.056,...,0.0898,0.372,86.919,audio_features,1j1Bn7kWnPWck79XYSTHI5,spotify:track:1j1Bn7kWnPWck79XYSTHI5,https://api.spotify.com/v1/tracks/1j1Bn7kWnPWc...,https://api.spotify.com/v1/audio-analysis/1j1B...,212093,4
23998,Justin Bieber,Beautiful Love - Free Fire,0Y1flwKfQc97a92yP7VbeW,0,2023-02-10,0,0.797,0.595,4,-7.482,...,0.1080,0.537,110.985,audio_features,0Y1flwKfQc97a92yP7VbeW,spotify:track:0Y1flwKfQc97a92yP7VbeW,https://api.spotify.com/v1/tracks/0Y1flwKfQc97...,https://api.spotify.com/v1/audio-analysis/0Y1f...,207838,4


In [103]:
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       1000 non-null   object 
 1   track_name        1000 non-null   object 
 2   track_id          1000 non-null   object 
 3   popularity        1000 non-null   int64  
 4   index             1000 non-null   int64  
 5   danceability      1000 non-null   float64
 6   energy            1000 non-null   float64
 7   key               1000 non-null   int64  
 8   loudness          1000 non-null   float64
 9   mode              1000 non-null   int64  
 10  speechiness       1000 non-null   float64
 11  acousticness      1000 non-null   float64
 12  instrumentalness  1000 non-null   float64
 13  liveness          1000 non-null   float64
 14  valence           1000 non-null   float64
 15  tempo             1000 non-null   float64
 16  type              1000 non-null   object 
 

In [79]:
concat_df.to_csv('spotify_pop_music.csv', index=False)

In [77]:
concat_df.columns

Index(['artist_name', 'track_name', 'track_id', 'popularity', 'release_date',
       'index', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature'],
      dtype='object')

In [107]:
concat_df.sample(5).iloc[:, [0, 1, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]]

,artist_name,track_name,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
456,Justin Bieber,Peaches (feat. Daniel Caesar & Giveon),81,0.677,0.696,0,-6.181,1,0.1190,0.3210,0.000000,0.420,0.464,90.030
983,Coldplay,Viva La Vida,0,0.486,0.617,5,-7.115,0,0.0287,0.0954,0.000003,0.109,0.417,138.015
146,Taylor Swift,cardigan,81,0.613,0.581,0,-8.588,0,0.0424,0.5370,0.000345,0.250,0.551,130.033
299,Doja Cat,Kiss Me More (feat. SZA),78,0.764,0.705,8,-3.463,1,0.0284,0.2590,0.000089,0.120,0.781,110.970
7,Miguel,Sure Thing,90,0.684,0.607,11,-8.127,0,0.1000,0.0267,0.000307,0.191,0.498,81.001
